# OpenStreetMap Case Study
----
Map Area:

Greater London

Link: https://mapzen.com/data/metro-extracts/your-extracts/c14328a6d43e

I chose Greater London because my home city Beirut data is below 50 MB and I will be visiting London for the first time next week for a vacation with my wife :)

# Problems encountered in my map
---
After running sample of my data against data.py from the case study, I noticed the below problems:
- Abbreviation of Saint to St in addr:Street
- Tags with key 'fix me' to be ignored
- Inconsistent Home numbers with comma and hyphens, convert comma to hyphen
- In Wikipedia, remove additional character 'en:' from string

### Fixing the St abbreviations in the key column

In [ ]:
# What I did here is I amended the update_name function we wrote earlier as below and fixed the regex formula
# For example it change 137,139 to 137-139 which is consistent with the rest of the housenumber of multiple
street_type_re = re.compile(r'^\b\S+\.?', re.IGNORECASE)

expected = ["Saint"]

mapping = { "St": "Saint",
            "st": "Saint"}

# I added the below inside my shape_element function
if value["k"] == 'addr:street':
    # After searching inside the street names
    m = ABBREVIATED_ST.search(value['v'])
    # Run agaianst the regex
    if m:
        street_type = m.group()
        if street_type in mapping.keys():
            # Run keys against the mapping dictionary identified above
            node_way_tags['value'] = re.sub(ABBREVIATED_ST, mapping[m.group()], value['v'])
            # Finally update any name

### Dropping all tags with keys "fixme" as they are not useful

In [ ]:
# Inside the data.py, I added the values fixme and FIXME to the keys to be ignored as I did with PROBLEMCHARS
if PROBLEMCHARS.search(value['k']) or (value['k']) == 'fixme' or (value['k']) == 'FIXME': 
            continue

### Inconsistent Home numbers with comma and hyphens, convert comma to hyphen

In [ ]:
elif value["k"] == 'addr:housenumber':
            m = STREETNAMES.search(value['v'])
            if m:
                node_way_tags['value'] = re.sub(',', '-', value['v'])

### In Wikipedia, remove additional character 'en:' from string

In [ ]:
# If key is wikipedia, I cleaned the value of the string from 'en:'
elif value["k"] == 'wikipedia':
            value_of_k = value['v']
            node_way_tags['value'] = value_of_k[3:]
            # Since it is fixed space I used index to slice it

## Converting CSV tables to SQLITE Databse
---

In [102]:
# The below is copied from this post: 
# https://discussions.udacity.com/t/creating-db-file-from-csv-files-with-non-ascii-unicode-characters/174958/8
# http://www.bogotobogo.com/python/python_sqlite_connect_create_drop_table.php

import sqlite3
import csv
from pprint import pprint

mydb = 'greater_london.db'    # name of the sqlite database file

# Connect to the database
conn = sqlite3.connect(mydb)

# Get a cursor object
cur = conn.cursor()

##############################################---NODES_TAGS---#########################################################

cur.execute('''DROP TABLE IF EXISTS nodes_tags''')
conn.commit()

# Create the nodes_tags table:
cur.execute('''
    CREATE TABLE nodes_tags (
    id INTEGER,
    key TEXT,
    value TEXT,
    type TEXT,
    FOREIGN KEY (id) REFERENCES nodes(id))
''')
# commit the changes
conn.commit()

# Read in the csv file as a dictionary, format the
# data as a list of tuples:
with open('nodes_tags.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'].decode("utf-8"), i['key'].decode("utf-8"),i['value'].decode("utf-8"), i['type'].decode("utf-8")) for i in dr]

# insert the formatted data
cur.executemany("INSERT INTO nodes_tags(id, key, value,type) VALUES (?, ?, ?, ?);", to_db)
# commit the changes
conn.commit()

##############################################---WAYS_TAGS---##########################################################

cur.execute('''DROP TABLE IF EXISTS ways_tags''')
conn.commit()

# Create the table, specifying the column names and data types:
cur.execute('''
    CREATE TABLE ways_tags (
    id INTEGER NOT NULL,
    key TEXT NOT NULL,
    value TEXT NOT NULL,
    type TEXT,
    FOREIGN KEY (id) REFERENCES ways(id))
''')
# commit the changes
conn.commit()

# Read in the csv file as a dictionary, format the
# data as a list of tuples:
with open('ways_tags.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'].decode("utf-8"), i['key'].decode("utf-8"),i['value'].decode("utf-8"), i['type'].decode("utf-8")) for i in dr]

# insert the formatted data
cur.executemany("INSERT INTO ways_tags(id, key, value,type) VALUES (?, ?, ?, ?);", to_db)
# commit the changes
conn.commit()

##############################################---NODES---##############################################################

cur.execute('''DROP TABLE IF EXISTS nodes''')
conn.commit()

# Create the table, specifying the column names and data types:
cur.execute('''
    CREATE TABLE nodes (
    id INTEGER PRIMARY KEY NOT NULL,
    lat REAL,
    lon REAL,
    user TEXT,
    uid INTEGER,
    version INTEGER,
    changeset INTEGER,
    timestamp TEXT)
''')
# commit the changes
conn.commit()

# Read in the csv file as a dictionary, format the
# data as a list of tuples:
with open('nodes.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'].decode("utf-8"), i['version'].decode("utf-8"), i['changeset'].decode("utf-8"), i['timestamp'].decode("utf-8"), i['user'].decode("utf-8"), i['uid'].decode("utf-8"), i['lat'].decode("utf-8"), i['lon'].decode("utf-8")) for i in dr]

# insert the formatted data
cur.executemany("INSERT INTO nodes(id, version, changeset, timestamp, user, uid, lat, lon) VALUES (?, ?, ?, ?, ?, ?, ?, ?);", to_db)
# commit the changes
conn.commit()

##############################################---WAYS---###############################################################

cur.execute('''DROP TABLE IF EXISTS ways''')
conn.commit()

# Create the table, specifying the column names and data types:
cur.execute('''
    CREATE TABLE ways (
    id INTEGER PRIMARY KEY NOT NULL,
    user TEXT,
    uid INTEGER,
    
    version TEXT,
    changeset INTEGER,
    timestamp TEXT)
''')
# commit the changes
conn.commit()

# Read in the csv file as a dictionary, format the
# data as a list of tuples:
with open('ways.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'].decode("utf-8"), i['version'].decode("utf-8"), i['changeset'].decode("utf-8"), i['timestamp'].decode("utf-8"), i['user'].decode("utf-8"), i['uid'].decode("utf-8")) for i in dr]

# insert the formatted data
cur.executemany("INSERT INTO ways(id, version, changeset, timestamp, user, uid) VALUES (?, ?, ?, ?, ?, ?);", to_db)
# commit the changes
conn.commit()

##############################################---WAYS_NODES---#########################################################

cur.execute('''DROP TABLE IF EXISTS ways_nodes''')
conn.commit()

# Create the table, specifying the column names and data types:
cur.execute('''
    CREATE TABLE ways_nodes (
    id INTEGER NOT NULL,
    node_id INTEGER NOT NULL,
    position INTEGER NOT NULL,
    FOREIGN KEY (id) REFERENCES ways(id),
    FOREIGN KEY (node_id) REFERENCES nodes(id))
''')
# commit the changes
conn.commit()

# Read in the csv file as a dictionary, format the
# data as a list of tuples:
with open('ways_nodes.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'].decode("utf-8"), i['node_id'].decode("utf-8"), i['position'].decode("utf-8")) for i in dr]

# insert the formatted data
cur.executemany("INSERT INTO ways_nodes(id, node_id, position) VALUES (?, ?, ?);", to_db)
# commit the changes
conn.commit()

# conn.close()

### Size of the file

In [103]:
import os
osm = os.path.getsize('greater_london.osm')/1000000
mydb = os.path.getsize('greater_london.db')/1000000
nodes_db = os.path.getsize('nodes.csv')/1000000
ways_db = os.path.getsize('ways.csv')/100000
nodes_tags_db = os.path.getsize('nodes_tags.csv')/100000
ways_tags_db = os.path.getsize('ways_tags.csv')/1000000
ways_nodes_db = os.path.getsize('ways_nodes.csv')/1000000

print 'greater_london.osm .........', osm, 'MB'
print 'greater_london.db ..........', mydb, 'MB'
print 'nodes.csv ...................', nodes_db, 'MB'
print 'ways.csv ....................', ways_db, 'MB'
print 'nodes_tags.csv ..............', nodes_tags_db, 'MB'
print 'ways_tags.csv ...............', ways_tags_db, 'MB'
print 'ways_nodes.csv ..............', ways_nodes_db, 'MB'

greater_london.osm ......... 52 MB
greater_london.db .......... 29 MB
nodes.csv ................... 16 MB
ways.csv .................... 22 MB
nodes_tags.csv .............. 38 MB
ways_tags.csv ............... 4 MB
ways_nodes.csv .............. 6 MB


### Number of unique users

In [104]:
cur.execute('SELECT COUNT(DISTINCT(e.uid))\
            FROM (SELECT uid FROM nodes\
            UNION ALL SELECT uid FROM ways) e')
cur.fetchall()

[(1940,)]

### Number of nodes

In [105]:
cur.execute('SELECT count(*)\
            FROM nodes')
cur.fetchall()

[(197114,)]

### Number of ways

In [106]:
cur.execute('SELECT count(*)\
            FROM ways')
cur.fetchall()

[(37393,)]

### Number of unique node tags types

In [107]:
cur.execute('SELECT COUNT(DISTINCT(key))\
            FROM nodes_tags')
cur.fetchall()

[(468,)]

### Number of unique node tags types

In [108]:
cur.execute('SELECT COUNT(DISTINCT(key))\
            FROM ways_tags')
cur.fetchall()

[(563,)]

### Unique number of types of Amenities

In [109]:
cur.execute('SELECT value\
            FROM nodes_tags\
            WHERE key = "amenity"\
            GROUP BY value')
len(cur.fetchall())

104

### Top 10 Amenities in nodes

In [170]:
cur.execute('SELECT value, count(value) AS Counter\
            FROM nodes_tags\
            WHERE key = "amenity"\
            GROUP BY value\
            ORDER BY Counter DESC\
            LIMIT 10')
g = cur.fetchall()
for key, value in g:
    print('{}: {}'.format(key, value))

bicycle_parking: 507
bench: 466
restaurant: 368
post_box: 298
telephone: 290
cafe: 270
waste_basket: 248
pub: 166
fast_food: 151
atm: 103


### Top 10 Amenities in ways

In [112]:
cur.execute('SELECT value, count(value) AS Counter\
            FROM ways_tags\
            WHERE key = "amenity"\
            GROUP BY value\
            ORDER BY Counter DESC\
            LIMIT 10')
cur.fetchall()

[(u'parking', 270),
 (u'restaurant', 225),
 (u'cafe', 167),
 (u'pub', 136),
 (u'school', 126),
 (u'place_of_worship', 114),
 (u'fast_food', 91),
 (u'bank', 35),
 (u'bar', 31),
 (u'community_centre', 27)]

### Historic categories and numbers available in Greater London

In [169]:
cur.execute('SELECT s.value, COUNT(s.value)\
            FROM \
            (SELECT key, value FROM nodes_tags UNION ALL SELECT key, value FROM ways_tags) AS s\
            WHERE key = "historic"\
            GROUP BY s.value')
g = cur.fetchall()
for key, value in g:
    print('{}: {}'.format(key, value))

archaeological_site: 2
blue_plaque: 4
cannon: 2
castle: 14
citywalls: 3
fence: 1
footway: 1
icon: 1
industrial: 1
memorial: 79
monastery: 1
monument: 13
police_telephone: 1
relic: 1
retaining_wall: 4
roman_road: 22
ruins: 4
ship: 2
yes: 3


### Top 10 contributing users

In [168]:
cur.execute('SELECT s.user, count(s.user) as Counter\
            FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) AS s\
            GROUP BY s.user\
            ORDER BY Counter DESC\
            LIMIT 10')
g = cur.fetchall()
for key, value in g:
    print('{}: {}'.format(key, value))

Tom Chance: 23100
Paul The Archivist: 16674
Amaroussi: 13199
Ed Avis: 12128
Derick Rethans: 10707
abc26324: 9941
peregrination: 8529
Harry Wood: 7314
ecatmur: 6361
moyogo: 5543


In [115]:
# Total number of entries recorded in this database
cur.execute('SELECT SUM(counter)\
            FROM\
            (SELECT s.user, count(s.user) as Counter\
            FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) AS s\
            GROUP BY s.user\
            ORDER BY Counter DESC) AS t')
cur.fetchall()

[(234507,)]

In [116]:
# Total number of entries recorded in this database from top 10 contributers
cur.execute('SELECT SUM(counter)\
            FROM\
            (SELECT s.user, count(s.user) as Counter\
            FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) AS s\
            GROUP BY s.user\
            ORDER BY Counter DESC\
            LIMIT 10) AS t')
cur.fetchall()

[(113496,)]

In [117]:
cur.execute('SELECT COUNT(DISTINCT(s.uid))\
            FROM\
            (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) AS s')
cur.fetchall()

[(1940,)]

## Conclution on this section

#### Looking the above data and knowing that we have 
- Total entries 234,507 entries
- 113,496 of 234,507 are from the top 10 contributers which is almost 50% of the entries
From the total number of users:
- 10 are contributing 50% of data
- 1930 are contributing the resulting 50%

# Additional Problems

### After queiying the data, I noticed lots of new issues which I was initmidated to neglect but couldn't, among these issues that didn't show at the beginnning of wrangling this data set was the below 2 issues:

1. I can see lots of "randomjunk_bot" in keys which I need to drop from my datasets.
2. I can see that there is a lot of "en" in keys which causes the key to be useless

## Here is how I fixed both new issues

## Problem 1:
Remove records with this key "randomjunk_bot",
As I did before I added this to my python script next to the above to drop if the key has this exact value

In [118]:
# Drop all keys with these values as they are useless and will contaminate the data
        if PROBLEMCHARS.search(value['k']) or (value['k']) == 'fixme' or (value['k']) == 'FIXME' or (value['k']) == 'randomjunk_bot':
            continue

## Problem 2:
Replace "en" in key with the proper value after it and get the type to be regular. An example is "species:en" where the "en" becomes the key and species becomes the "type" which is not consistent with this type of data where species shall be the key with type regular

In [119]:
# Whenever the key is valued as en, replace it with the value in type and put type regular 
if node_way_tags['key'] == 'en':
    node_way_tags['key'] = node_way_tags['type']
    node_way_tags['type'] = 'regular'

## Number of unique species available in Greater London

In [120]:
cur.execute('SELECT COUNT(DISTINCT(value))\
            FROM nodes_tags\
            WHERE key = "species"')
cur.fetchall()

[(326,)]

# Issues that can be improved
Looking at the data for building, we can notice that below

In [151]:
# Number of entries with type building
cur.execute('SELECT count(value)\
            FROM ways_tags\
            WHERE key = "building"')
cur.fetchall()

[(19379,)]

In [223]:
# Types of building with counter
cur.execute('SELECT value, COUNT(value)\
            FROM ways_tags\
            WHERE key = "building"\
            GROUP BY value')
g = cur.fetchall()
for key, value in g:
    print('{}: {}'.format(key.upper(), value))

ETON_PLACE: 1
AIR_SHAFT: 14
APARTMENTS: 1077
BANDSTAND: 2
BLOCK: 5
BOAT: 1
BRIDGE: 1
BUS_GARAGE: 1
CASTLE: 13
CHAPEL: 1
CHIMNEY: 1
CHURCH: 34
CIVIC: 1
CLOCK_TOWER: 1
CLUBHOUSE: 1
COLLEGE: 4
COMMERCIAL: 219
CONSTRUCTION: 9
CONVENT: 1
COUNCIL_FLATS: 3
DATA_CENTER: 3
DEPOT: 1
DORMITORY: 1
FACADE: 1
FACULTY: 4
FARM: 1
FARM_AUXILIARY: 1
FERRY_TERMINAL: 1
FLATS: 24
GALLERY: 1
GARAGE: 39
GARAGES: 42
GASOMETER: 2
GRANDSTAND: 1
GREENHOUSE: 3
HALL_OF_RESIDENCE: 1
HOSPITAL: 12
HOTEL: 15
HOUSE: 3511
HOUSES: 2
HUT: 6
INDUSTRIAL: 69
KINDERGARTEN: 1
KIOSK: 1
LIGHT_INDUSTRIAL: 1
MOSQUE: 1
MULTIPLE: 1
NO: 14
OFFICE: 77
OFFICES: 3
OUTBUILDING: 23
PART: 12
PAVILION: 1
PLACE_OF_WORSHIP: 1
PORTACABIN: 1
PRISON: 1
PUB: 3
PUBLIC: 7
RESIDENTIAL: 1342
RETAIL: 228
ROOF: 37
SCHOOL: 70
SEMIDETACHED_HOUSE: 2
SERVICE: 1
SHED: 11
SHIP: 2
SHOP: 7
STADIUM: 1
STATION: 16
STUDENT_ACCOMODATION: 1
STUDENT_RESIDENCE: 1
SUBSTATION: 1
TERRACE: 361
TOWER: 6
TRAIN_STATION: 14
TUNNEL_ENTRANCE: 1
TUNNEL_SHAFT: 1
UNIVERSITY: 24
U

#### We have 19,379 entries with key building, but if we look closer we will notice that out of these records,  11,961 (almost 60%) are labeled "yes" which is good but not good enough knowing that the data can be more specific

### An improvement that can be made here is be more specific on the type of the building especially for tourists who might want to know if this building is a hotel or residential.

### I believe this is easier said than done, but looking above we notice that the majority of the constibuters on this dataset are less than maybe 30 users, for that we can use a rewarding system that might reward them points for their contributions on this issue and other issues.

# Additional issues I want to investigate

### Most popular type of shops in Greater London

In [217]:
cur.execute('SELECT s.value, COUNT(*) as counter \
            FROM nodes_tags AS s \
            JOIN (SELECT DISTINCT(id) FROM nodes_tags) AS t \
            ON s.id = t.id \
            WHERE s.key="shop" \
            GROUP BY s.value \
            ORDER BY counter DESC \
            LIMIT 1')
g = cur.fetchall()
for key, value in g:
    print('{}: {}'.format(key, value))

clothes: 152


### Top 10 type of shops in Greater London

In [221]:
cur.execute('SELECT s.value, COUNT(*) as counter \
            FROM nodes_tags AS s \
            JOIN (SELECT DISTINCT(id) FROM nodes_tags) AS t \
            ON s.id = t.id \
            WHERE s.key="shop" \
            GROUP BY s.value \
            ORDER BY counter DESC \
            LIMIT 10')
g = cur.fetchall()
for key, value in g:
    print('{}: {}'.format(key.upper(), value))

CLOTHES: 152
CONVENIENCE: 151
HAIRDRESSER: 78
SUPERMARKET: 45
ESTATE_AGENT: 42
JEWELRY: 33
BEAUTY: 31
DRY_CLEANING: 31
NEWSAGENT: 31
FURNITURE: 29


### Looking at the results, the data looks logical knowing that Greater London is renowned for its shopping experience and due to being the most expensive real estate in EU, having estate agent shops makes sense
